In [2]:
import numpy as np
from typing import Tuple, List
import logging

In [3]:
np.random.seed(2)


class mine_field:
    def __init__(self, x_dim, y_dim, mine_count):
        # Store parameters:
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.mine_count = mine_count

        # Generate matrix:
        self.reset_matrix()

    def reset_matrix(self):
        # Generating mine positions and coordinates:
        mine_positions = np.sort(
            np.random.randint(0, self.x_dim * self.y_dim, size=self.mine_count)
        )
        self.mine_coordinates = np.array(
            [(int(x / self.x_dim), x % self.x_dim) for x in mine_positions]
        ).tolist()
        logging.debug(self.mine_coordinates)

        # Generate matrix with Nan-s:
        self.matrix = np.full([self.y_dim, self.x_dim], np.nan)

        # Initialize clicked fields:
        self.clicked = []

    def set_cell_value(self, click: Tuple[int], value) -> None:
        self.matrix.itemset(click, value)

    def get_mine_count(self, neighbour_indices: np.array) -> int:
        mines_in_neighbours = 0
        for neighbour in neighbour_indices.tolist():
            if neighbour in self.mine_coordinates:
                mines_in_neighbours += 1
        return mines_in_neighbours

    def get_matrix(self):
        return self.matrix

    def get_all_neighbours(self, click: Tuple[int]) -> List[Tuple[int]]:
        (x_max, y_max) = self.matrix.shape

        neighbours: list = []
        for i in range(-1, 2):
            for j in range(-1, 2):
                # Let's skip the original coordinate:
                if j == 0 and i == 0:
                    continue

                # Calculating new coordinates:
                new_x = click[0] + i
                new_y = click[1] + j

                # Don't bother with negative coordaintes:
                if new_x < 0 or new_y < 0:
                    continue

                # Don't bother with coordaintes extending beyond the limits:
                if new_x >= x_max or new_y >= y_max:
                    continue

                neighbours.append([new_x, new_y])

        return np.array(neighbours)

    def plot_matrix_ascii(self):
        plot = ""
        for row in self.matrix:
            for item in row:
                if item == 9 or np.isnan(item):
                    plot += "X "
                else:
                    plot += f"{int(item)} "
            plot += "\n"

        print(plot)

    def click_on(self, click) -> int:
        logging.debug(f"clicking on: {click}")

        # Test if we have already clicked:
        if click in self.clicked:
            logging.info(f"The field: {click} is already clicked on.")
            return 2

        # Are we clicking on a mine:
        if list(click) in self.mine_coordinates:
            print("Game over. You clicked on a mine! :(")
            return 1

        # Adding click to list:
        self.clicked.append(click)

        # 1. Get all neighbours:
        neighbours = self.get_all_neighbours(click)

        # 1. Get number of mines around the cliced cell:
        mine_count = self.get_mine_count(neighbours)

        # 2. Update value of the matrix at index:
        self.set_cell_value(click, mine_count)

        # 3. Recursion: if the value is zero, click on all the neighbours:
        if mine_count == 0:
            for neighbour in neighbours:
                if tuple(neighbour) not in self.clicked:
                    self.click_on(tuple(neighbour))

        # 4. Testing if the number of unclicked fields is the same as the number of mines:
        if len(self.mine_coordinates) == (self.x_dim * self.y_dim) - len(self.clicked):
            print("Game over. You won! :)")
            return -1

        # If we didn't win with the last click, we just keep going:
        return 0


table = mine_field(10, 10, 10)
table.plot_matrix_ascii()

X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 
X X X X X X X X X X 



In [4]:
table.click_on((7, 7))
table.plot_matrix_ascii()

X X X X X X X X 1 0 
X X X X X X 2 1 1 0 
X X X X X 2 1 0 0 0 
X X X X X 1 0 0 1 1 
X X X X 2 1 0 0 1 X 
X X X X 1 0 0 0 1 1 
X X X X 1 1 1 0 0 0 
X X X X X X 1 0 0 0 
X X X 2 1 1 1 0 0 0 
X X X 1 0 0 0 0 0 0 



In [5]:
table.click_on((7, 5))
table.plot_matrix_ascii()

Game over. You clicked on a mine! :(
X X X X X X X X 1 0 
X X X X X X 2 1 1 0 
X X X X X 2 1 0 0 0 
X X X X X 1 0 0 1 1 
X X X X 2 1 0 0 1 X 
X X X X 1 0 0 0 1 1 
X X X X 1 1 1 0 0 0 
X X X X X X 1 0 0 0 
X X X 2 1 1 1 0 0 0 
X X X 1 0 0 0 0 0 0 

